环境用的：test

https://www.bilibili.com/video/BV1pfKNe8E7F/?spm_id_from=333.1007.tianma.3-4-10.click&vd_source=a0d30acd76c4a4ca61e1f938925ede3e

In [1]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install transformer
# !pip install datasets
# !pip install peft
# !pip install bitsandbytes

In [3]:
# 测试模型是否可用
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 运行通过了
# model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

print("----模型加载成功----")

/home/ubuntu/anaconda3/envs/test/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


----模型加载成功----


In [4]:
# 制作数据集
from data_prepare import samples
import json

with open("datasets.jsonl","w",encoding="utf-8") as f:
    for s in samples:
        json_line = json.dumps(s) # 将每行数据转换为json格式
        f.write(json_line + "\n") # 写入文件
    else:
        print("----数据准备完成----")

----数据准备完成----


In [5]:
# 准备训练集和测试集
from datasets import load_dataset

dataset = load_dataset("json",data_files={"train":"datasets.jsonl"},split="train")
print("数据量为：",len(dataset))

print("----数据集准备完成----")

# 分割数据集
train = dataset.train_test_split(test_size=0.2)

train_dataset = train["train"]
test_dataset = train["test"]

print("训练集数据量为：",len(train_dataset))
print("测试集数据量为：",len(test_dataset))

print("----数据集分割完成----")

Generating train split: 50 examples [00:00, 6269.14 examples/s]

数据量为： 50
----数据集准备完成----
训练集数据量为： 40
测试集数据量为： 10
----数据集分割完成----


In [6]:
# 编写tokenizer的处理工具
def tokenizer_function(examples):
    # 将 examples 字典中的 prompt 列表和 completion 列表中的对应元素配对。
    # 将每对 prompt 和 completion 拼接成一个字符串，并在它们之间插入一个换行符。
    # 将所有这些拼接后的字符串放入一个新的列表 texts 中。
    texts  = [f"{prompt}\n{completion}" for prompt, completion in zip(examples["prompt"],examples["completion"])]
    print(texts)

    tokens = tokenizer(texts, truncation=True, max_length=512, padding="max_length")
    # 将 tokens 字典中的 input_ids 列表复制到 label 列表中。
    tokens["label"] = tokens["input_ids"].copy() 

    return tokens

# batch 参数表示是否对数据进行批处理
tokenized_train_dataset = train_dataset.map(tokenizer_function,batched=True)
tokenized_test_dataset = test_dataset.map(tokenizer_function,batched=True)

print("----数据集tokenize完成----")

Map: 100%|██████████| 40/40 [00:00<00:00, 779.95 examples/s]


['Question 40: Why is it important to rest your voice after singing?\nAnswer 40: Resting allows your vocal cords to recover, preventing strain and potential long-term damage.', 'Question 12: How do you sing high notes without straining?\nAnswer 12: Use proper breath support, relax your throat, and place the sound forward in your mask (resonance areas).', 'Question 9: What techniques help develop vibrato in singing?\nAnswer 9: Practice sustaining notes while gently oscillating your pitch; focus on relaxation and consistent airflow.', 'Question 50: What is the final step in preparing for a vocal performance?\nAnswer 50: Do a final warm-up, review your setlist, and focus on calming your nerves before stepping onto the stage.', 'Question 23: How do you project your voice without shouting?\nAnswer 23: Use proper breath support, open your throat, and focus on forward resonance to amplify your sound naturally.', 'Question 34: What exercises help with vocal endurance?\nAnswer 34: Long-tone exe

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

['Question 20: How do you develop confidence in your singing voice?\nAnswer 20: Record and listen to your progress, focus on your strengths, and perform regularly to build confidence over time.', 'Question 49: How do you handle feedback as a singer?\nAnswer 49: Listen to feedback with an open mind, focus on constructive criticism, and use it as an opportunity to grow.', 'Question 8: How do you improve vocal tone quality?\nAnswer 8: Focus on resonance and placement, and practice vocal exercises like sirens to achieve a smoother tone.', 'Question 39: How do you prepare for a live singing performance?\nAnswer 39: Warm up thoroughly, rehearse your setlist, and visualize a successful performance to build confidence.', 'Question 48: What is the importance of storytelling in singing?\nAnswer 48: Storytelling connects you with the audience, making your performance more impactful and memorable.', 'Question 42: What are the benefits of singing scales daily?\nAnswer 42: Daily scales improve pitch

Map: 100%|██████████| 10/10 [00:00<00:00, 372.94 examples/s]

----数据集tokenize完成----


In [7]:
# 量化设置
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,   # 加载8位模型
)

model = AutoModelForCausalLM.from_pretrained(model_name,quantization_config=quantization_config, device_map="auto")

print("----模型量化完成----")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


----模型量化完成----


In [8]:
# Lora 微调设置
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=8,  # Lora 矩阵的秩（rank）。秩决定了 Lora 矩阵的维度，通常是一个较小的整数。较小的秩可以减少计算量，但可能会影响模型的性能。
    lora_alpha=16,  # Lora 矩阵的缩放因子。这个参数用于调整 Lora 矩阵的权重，通常是一个较大的整数。较大的值可以增强 Lora 矩阵的影响。
    lora_dropout=0.05,  # Lora 矩阵的 dropout 概率。dropout 是一种正则化技术，用于防止过拟合。这个参数指定了在训练过程中随机丢弃 Lora 矩阵中某些元素的概率。
    task_type=TaskType.CAUSAL_LM,  # 任务类型。`TaskType.CAUSAL_LM` 表示这是一个因果语言建模任务，即预测下一个词的任务。
)

model = get_peft_model(model, lora_config)
print("------------------------")
model.print_trainable_parameters()

print("----Lora 微调设置完成----")

------------------------
trainable params: 1,089,536 || all params: 1,778,177,536 || trainable%: 0.0613
----Lora 微调设置完成----


In [ ]:
# 设置训练参数
from transformers import TrainingArguments,Trainer

trainning_args = TrainingArguments(
    output_dir="./main_results",
    num_train_epochs=50, 
    per_device_train_batch_size=4, # 每个GPU的batch size
    gradient_accumulation_steps=8, # 梯度累积步数
    fp16=True,
    logging_steps=10, # 每多少步保存一次日志
    save_steps=10, # 每多少步保存一次模型
    eval_strategy='steps', # 评估策略，steps表示每多少步评估一次
    eval_steps=10, # 每多少步评估一次
    learning_rate=5e-5,
    logging_dir='./main_logs',
    run_name = "Deepseek_main_finetune" # 运行名称
)

print("----训练参数设置完成----")

----训练参数设置完成----


In [11]:
# 定义训练器
trainer = Trainer(
    model=model,
    args=trainning_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
)

print("----训练器定义完成----")

trainer.train()

print("----训练完成----")

/tmp/ipykernel_274506/3575231236.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


----训练器定义完成----


Step,Training Loss,Validation Loss
10,5.563600,4.314012
20,2.915300,1.672166
30,1.016300,0.528060
40,0.400200,0.310636
50,0.316400,0.295170
60,0.304100,0.289939
70,0.304300,0.285583
80,0.301100,0.283222
90,0.291500,0.281372
100,0.296500,0.281480


/home/ubuntu/anaconda3/envs/test/lib/python3.12/site-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fe11cbd7200>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 7a237fed-7ac6-4849-9130-d01cb5d7f607)') - silently ignoring the lookup for the file config.json in deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B.
  warnings.warn(
/home/ubuntu/anaconda3/envs/test/lib/python3.12/site-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B - will assume that the vocabulary was not modified.
  warnings.warn(
/home/ubuntu/anaconda3/envs/test/lib/python3.12/site-packages/peft/utils/other.py:716: Use

----训练完成----


In [ ]:
# 保存lora模型
save_path = "./main_results/FT-result"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("----模型保存完成----")

/home/ubuntu/anaconda3/envs/test/lib/python3.12/site-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fe11cbba540>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: a246f428-d2a3-4387-98fb-759b73f5453f)') - silently ignoring the lookup for the file config.json in deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B.
  warnings.warn(
/home/ubuntu/anaconda3/envs/test/lib/python3.12/site-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B - will assume that the vocabulary was not modified.
  warnings.warn(


----模型保存完成----


In [13]:
# 保存全量模型
final_save_path = "./main_results/FT-result-full"


from peft import PeftModel
base_model = AutoModelForCausalLM.from_pretrained(model_name)
model = PeftModel.from_pretrained(base_model,save_path)
model = model.merge_and_unload()

model.save_pretrained(final_save_path)
tokenizer.save_pretrained(final_save_path)

print("----全量模型保存完成----")

----全量模型保存完成----


In [17]:
## 调用模型进行推理
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

final_save_path = "./main_results/FT-result-full"
model = AutoModelForCausalLM.from_pretrained(final_save_path)
tokenizer = AutoTokenizer.from_pretrained(final_save_path)

print("----模型加载完成----")

# 构建推理pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

print("----推理pipeline构建完成----")

# 进行推理
prompt = "tell me some singing skills"
output = pipe(prompt, max_length=512, num_return_sequences=1)

print("----推理完成----")
print(output)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.99it/s]
Device set to use cuda:0


----模型加载完成----


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


----推理pipeline构建完成----
----推理完成----
[{'generated_text': "tell me some singing skills that I can practice in order to improve my singing.\nOkay, so I want to improve my singing skills. I've heard that practicing a lot can help, but I'm not sure where to start. Maybe I should look into singing along to music or something. But I'm not sure how to do that properly. I've also heard about singing vocabulary being important, but I don't know much about it. Maybe I should figure out what words I can sing and how to use them. Also, I think practicing different instruments can help me get better, but I don't know much about that. Maybe I should look into learning how to play an instrument or use different instruments in my singing. I'm not sure if I want to sing in a band or just sing on my own. Maybe I should start with something easier and build up from there. I'm also a bit worried about making mistakes. I don't want to end up with a song that's not good, so I need to learn how to improve fro

{"prompt": "Question 4: How do you find your vocal range?", "completion": "Answer 4: Experiment by singing scales up and down to identify the lowest and highest note you can comfortably sustain."}